# Actor-Critic

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [Q^\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)]$$

Встает вопрос, как оценить $Q^\pi(s, a)$? В чистом policy-based алгоритме REINFORCE используется отдача $G_t$, полученная методом Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$. В Actor-Critic же предлагается отдельно обучать нейронную сеть Q-функции — критика.

Актор-критиком часто называют обобщенный фреймворк (подход), нежели какой-то конкретный алгоритм. Как подход актор-критик не указывает, каким конкретно [policy gradient] методом обучается актор и каким [value based] методом обучается критик. Таким образом актор-критик задает целое [семейство](https://proceedings.neurips.cc/paper_files/paper/1999/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf) различных алгоритмов. Рекомендую в качестве шпаргалки использовать упомянутый в тетрадке с REINFORCE [пост из блога Lilian Weng](https://lilianweng.github.io/posts/2018-04-08-policy-gradient/), посвященный наиболее популярным алгоритмам семейства актор-критиков

В данной тетрадке познакомимся с наиболее простым вариантом актор-критика, который так и называют Actor-Critic:

In [1]:
# Cтавим нужные зависимости, если это колаб
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [2]:
from collections import deque
import random

import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.distributions import Categorical

%matplotlib inline

In [3]:
env = gym.make("CartPole-v1")
env.reset()

print(f'{env.observation_space=}')
print(f'{env.action_space=}')

n_actions = env.action_space.n
state_dim = env.observation_space.shape
print(f'Action_space: {n_actions} | State_space: {env.observation_space.shape}')

env.observation_space=Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
env.action_space=Discrete(2)
Action_space: 2 | State_space: (4,)


(1 балл)

In [4]:
def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x)
    return x

def symlog(x):
    """Compute symlog values for a vector `x`. It's an inverse operation for symexp."""
    return torch.sign(x) * torch.log(torch.abs(x) + 1)

def symexp(x):
    """Compute symexp values for a vector `x`. It's an inverse operation for symlog."""
    return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)


class SymExpModule(nn.Module):
    def forward(self, x):
        return symexp(x)

def select_action_eps_greedy(Q, state, epsilon):
    """Выбирает действие epsilon-жадно."""
    if not isinstance(state, torch.Tensor):
        state = torch.tensor(state, dtype=torch.float32)
    Q_s = Q(state).detach().numpy()

    if np.random.uniform(0, 1) < epsilon:
        num_actions = Q_s.shape[-1]
        action = np.random.randint(0, num_actions)
    else:
        action = np.argmax(Q_s)

    action = int(action)
    return action

def sample_batch(replay_buffer, n_samples):
    # sample randomly `n_samples` samples from replay buffer
    # and split an array of samples into arrays: states, actions, rewards, next_actions, terminateds
    batch_data = random.sample(replay_buffer, n_samples)
    states, actions, rewards, next_states, terminateds = zip(*batch_data)

    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(terminateds)

## Shared-body Actor-Critic

Актор и критик могут обучаться в разных режимах — актор только on-policy (шаг обучения на текущей собранной подтраектории), а критик on-policy или off-policy (шаг обучения на текущей подтраектории или на батче из replay buffer). Это с одной стороны привносит гибкость в обучение, с другой — усложняет его.

Если актор и критик оба обучаются on-policy, то имеет смысл объединить их сетки в одну и делать общий шаг обратного распространения ошибки. Однако, если они обучаются в разных режимах (и с разной частотой обновления), то велика вероятность, что их шаги обучения могут начать конфликтовать в случае общего тела — для такого варианта намного предпочтительнее разделить их на разные подсети (либо аккуратно настраивать гиперпарметры, чтобы стабилизировать обучение). В целом, рекомендуется использовать общий энкодер наблюдений, а далее как можно скорее разделять головы.

Сделаем реализацию актор-критика с общим телом и с on-policy вариантом обучения.

In [5]:
class ActorBatch:
    def __init__(self):
        self.logprobs = []
        self.q_values = []

    def append(self, log_prob, q_value):
        self.logprobs.append(log_prob)
        self.q_values.append(q_value)

    def clear(self):
        self.logprobs.clear()
        self.q_values.clear()

(3 балла)

In [6]:
class ActorCriticModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()

        # Инициализируйте сеть агента с двумя головами: softmax-актора и линейного критика
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dims[0]))
        layers.append(nn.ReLU())
        for i in range(len(hidden_dims) - 1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.ReLU())
        self.net = nn.Sequential(*layers)
                
        self.actor_head = nn.Sequential(
            nn.Linear(hidden_dims[-1], output_dim),
            nn.Softmax(dim=-1)
        )
        self.critic_head = nn.Linear(hidden_dims[-1], 1)

    def forward(self, state):
        features = self.net(state)
        action_probs = self.actor_head(features)
        
        distribution = Categorical(action_probs)
        action = distribution.sample()
        log_prob = distribution.log_prob(action)
        
        state_value = self.critic_head(features)
        
        return action.unsqueeze(1), log_prob.unsqueeze(1), state_value

    def evaluate(self, state):
        features = self.net(state)
        return self.critic_head(features)

(6 баллов)

In [7]:
class ActorCriticAgent:
    def __init__(self, state_dim, action_dim, hidden_dims, lr, gamma, critic_rb_size):
        self.lr = lr
        self.gamma = gamma

        # Инициализируйте модель актор-критика и SGD оптимизатор (например, `torch.optim.Adam)`)
        self.actor_critic = ActorCriticModel(state_dim, hidden_dims, action_dim)
        self.opt = torch.optim.Adam(self.actor_critic.parameters(), lr=self.lr)
        self.actor_batch = ActorBatch()
        self.critic_rb = deque(maxlen=critic_rb_size)

    def act(self, state):
        # Произведите выбор действия и сохраните необходимые данные в батч для последующего обучения
        state_tensor = to_tensor(state).unsqueeze(0)
        action_tensor, log_prob, state_value = self.actor_critic(state_tensor)
        self.actor_batch.append(log_prob, state_value.detach())

        return action_tensor.item()

    def append_to_replay_buffer(self, s, a, r, next_s, terminated):
        # Добавьте новый экземпляр данных в память прецедентов.
        transition = (s, a, r, next_s, terminated)
        self.critic_rb.append(transition)
        
    def evaluate(self, state):
        return self.actor_critic.evaluate(state)

    def update(self, rollout_size, critic_batch_size, critic_updates_per_actor):
        if len(self.actor_batch.q_values) < rollout_size:
            return

        self.opt.zero_grad()
        critic_loss = self.update_critic(critic_batch_size, critic_updates_per_actor)
        actor_loss = self.update_actor()
        total_loss = critic_loss + actor_loss
        
        if total_loss.requires_grad:
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.actor_critic.parameters(), max_norm=1.0)
            self.opt.step()
        
        self.actor_batch.clear()

    def update_actor(self):
        state_values = to_tensor(self.actor_batch.q_values)
        logprobs = torch.stack(self.actor_batch.logprobs)

        s, a, r, s_next, term = zip(*self.critic_rb)
        
        rewards = to_tensor(r)
        next_states = to_tensor(s_next)
        terminated = to_tensor(term, float)
        
        with torch.no_grad():
            next_state_values = self.actor_critic.evaluate(next_states).squeeze()
            # TD target для value function
            targets = rewards + self.gamma * next_state_values * (1 - terminated)
        
        # Advantage = TD error
        advantages = targets - state_values.squeeze()
        
        # Policy gradient loss
        actor_loss = -(logprobs.squeeze() * advantages.detach()).mean()
        
        return actor_loss

    def update_critic(self, batch_size, n_updates=1):
        # Реализуйте n_updates шагов обучения критика.
        total_critic_loss = 0.0
        for _ in range(n_updates):
            if len(self.critic_rb) < batch_size:
                continue
            states, actions, rewards, next_states, terminateds = sample_batch(self.critic_rb, batch_size)
            td_loss = self.compute_td_loss(states, actions, rewards, next_states, terminateds)
            total_critic_loss += td_loss
            
        return total_critic_loss

    def compute_td_loss(
        self, states, actions, rewards, next_states, terminateds, regularizer=0.01
    ):
        s = to_tensor(states)
        r = to_tensor(rewards)
        s_next = to_tensor(next_states)
        term = to_tensor(terminateds, float)

        current_values = self.evaluate(s).squeeze()
        
        with torch.no_grad():
            next_values = self.evaluate(s_next).squeeze()
            targets = r + self.gamma * next_values * (1 - term)
        
        td_error = targets - current_values
        
        # MSE loss для value function
        loss = torch.mean(td_error ** 2)
        loss += regularizer * torch.mean(current_values ** 2)
        return loss

In [8]:
def run_actor_critic(
        env_name="CartPole-v1",
        hidden_dims=(128, 128), lr=5e-4,
        total_max_steps=200_000,
        train_schedule=16, replay_buffer_size=50000, batch_size=64, critic_updates_per_actor=4,
        eval_schedule=1000, smooth_ret_window=10, success_ret=200.
):
    env = gym.make(env_name)
    episode_return_history = deque(maxlen=smooth_ret_window)

    agent = ActorCriticAgent(
        state_dim=env.observation_space.shape[0], action_dim=env.action_space.n, hidden_dims=hidden_dims,
        lr=lr, gamma=.995, critic_rb_size=replay_buffer_size
    )

    s, _ = env.reset()
    done, episode_return = False, 0.
    eval = False

    for global_step in range(1, total_max_steps+1):
        a = agent.act(s)
        s_next, r, terminated, truncated, _ = env.step(a)
        episode_return += r
        done = terminated or truncated

        # train step
        agent.append_to_replay_buffer(s, a, r, s_next, terminated)
        agent.update(train_schedule, batch_size, critic_updates_per_actor)

        # evaluate
        if global_step % eval_schedule == 0:
            eval = True

        s = s_next
        if done:
            if eval:
                episode_return_history.append(episode_return)
                avg_return = np.mean(episode_return_history)
                print(f'{global_step=} | {avg_return=:.3f}')
                if avg_return >= success_ret:
                    print('Решено!')
                    break

            s, _ = env.reset()
            done, episode_return = False, 0.
            eval = False

run_actor_critic(
    eval_schedule=2000, 
    total_max_steps=100_000,
    lr=1e-3,
    batch_size=16,
    train_schedule=16,
    replay_buffer_size=16
)

global_step=2004 | avg_return=12.000
global_step=4009 | avg_return=13.000
global_step=6021 | avg_return=43.000
global_step=8036 | avg_return=52.250
global_step=10026 | avg_return=50.600
global_step=12002 | avg_return=43.833
global_step=14011 | avg_return=43.857
global_step=16020 | avg_return=49.375
global_step=18119 | avg_return=59.556
global_step=20028 | avg_return=57.200
global_step=22021 | avg_return=58.600
global_step=24097 | avg_return=76.300
global_step=26018 | avg_return=79.600
global_step=28215 | avg_return=104.600
global_step=30177 | avg_return=120.000
global_step=32052 | avg_return=142.600
global_step=34139 | avg_return=154.200
global_step=36146 | avg_return=162.000
global_step=38148 | avg_return=163.200
global_step=40160 | avg_return=197.800
global_step=42018 | avg_return=215.500
Решено!
